In [1]:
from datasets import load_dataset
from training_datasets.GLUEDataset import GLUEDataset

cola = load_dataset('glue','cola')
# print(cola)
# sst2 = load_dataset('glue','sst2')
# print(sst2)
mrpc = load_dataset('glue', 'mrpc')
# print(mrpc)
pt_cola = GLUEDataset([cola, mrpc], 'train')

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1004.46it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1419.87it/s]


In [2]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

# sampler = FewShotEpisodeSampler(pt_cola, kShot=2, nWay=4, shuffle=True)
# train_data_loader = data.DataLoader(
#     pt_cola,
#     batch_sampler=sampler,
#     num_workers=1,
# )

# data, targets = next(iter(train_data_loader))

# for i in range(5):
#     data, targets = next(iter(train_data_loader))
#     print(data)
#     print(targets)

train_protomaml_sampler = FewShotEpisodeBatchSampler(pt_cola, kShot=3, nWay=4, batchSize=8, shuffle=True)
train_protomaml_loader = data.DataLoader(
    pt_cola, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=1
)

# for i in range(1):
#     batch = next(iter(train_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)

In [3]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [4]:
from training.models.ProtoFOMAML import ProtoFOMAML

pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
pfomaml.training_step(next(iter(train_protomaml_loader)), 0)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relati

losses are tensor([1.1001, 1.1574, 1.0943, 1.0817, 1.1202, 1.1844],
       grad_fn=<NllLossBackward0>) and loss is 6.738034248352051
losses are tensor([1.0504, 1.1269, 1.0447, 1.0479, 1.0618, 1.1556],
       grad_fn=<NllLossBackward0>) and loss is 6.487463474273682
losses are tensor([1.0056, 1.0982, 1.0023, 1.0167, 1.0097, 1.1286],
       grad_fn=<NllLossBackward0>) and loss is 6.261122703552246
losses are tensor([0.9645, 1.0712, 0.9641, 0.9872, 0.9620, 1.1034],
       grad_fn=<NllLossBackward0>) and loss is 6.052452087402344
losses are tensor([0.9280, 1.0454, 0.9306, 0.9593, 0.9197, 1.0794],
       grad_fn=<NllLossBackward0>) and loss is 5.862411022186279
losses are tensor([0.8569, 1.1338, 0.9527, 1.3150, 1.1297, 1.1033],
       grad_fn=<NllLossBackward0>) and loss is 6.49141788482666
losses are tensor([0.7838, 1.0822, 0.8850, 1.2682, 1.0851, 1.0573],
       grad_fn=<NllLossBackward0>) and loss is 6.161527633666992
losses are tensor([0.7198, 1.0343, 0.8254, 1.2256, 1.0439, 1.0170],
  

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relati

losses are tensor([1.2010, 1.0766, 1.1957, 1.0548, 1.1543, 1.1092],
       grad_fn=<NllLossBackward0>) and loss is 6.791520595550537
losses are tensor([1.1651, 1.0390, 1.1543, 1.0097, 1.1080, 1.0720],
       grad_fn=<NllLossBackward0>) and loss is 6.5482177734375
losses are tensor([1.1323, 1.0044, 1.1166, 0.9685, 1.0664, 1.0376],
       grad_fn=<NllLossBackward0>) and loss is 6.325778961181641
losses are tensor([1.1029, 0.9724, 1.0814, 0.9307, 1.0289, 1.0053],
       grad_fn=<NllLossBackward0>) and loss is 6.1215338706970215
losses are tensor([1.0754, 0.9436, 1.0495, 0.8951, 0.9938, 0.9768],
       grad_fn=<NllLossBackward0>) and loss is 5.934208869934082
losses are tensor([1.2184, 1.0368, 1.1296, 1.0396, 1.0793, 0.9988],
       grad_fn=<NllLossBackward0>) and loss is 6.502643585205078
losses are tensor([1.1732, 0.9748, 1.0725, 0.9757, 1.0244, 0.9556],
       grad_fn=<NllLossBackward0>) and loss is 6.176210403442383
losses are tensor([1.1293, 0.9174, 1.0185, 0.9163, 0.9714, 0.9140],
  

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relati

...

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relati

losses are tensor([1.2302, 1.1767, 1.1402, 1.0605, 1.1634, 1.1744],
       grad_fn=<NllLossBackward0>) and loss is 6.945544719696045
losses are tensor([1.1802, 1.1276, 1.0822, 1.0030, 1.1151, 1.1276],
       grad_fn=<NllLossBackward0>) and loss is 6.6356329917907715
losses are tensor([1.1351, 1.0818, 1.0289, 0.9506, 1.0702, 1.0850],
       grad_fn=<NllLossBackward0>) and loss is 6.351590156555176
losses are tensor([1.0941, 1.0391, 0.9796, 0.9028, 1.0282, 1.0458],
       grad_fn=<NllLossBackward0>) and loss is 6.089634895324707
losses are tensor([1.0566, 0.9992, 0.9336, 0.8590, 0.9894, 1.0096],
       grad_fn=<NllLossBackward0>) and loss is 5.84744119644165
losses are tensor([1.1556, 1.1550, 1.1271, 1.2184, 1.1829, 1.1523],
       grad_fn=<NllLossBackward0>) and loss is 6.991264343261719
losses are tensor([1.1184, 1.1146, 1.0790, 1.1639, 1.1486, 1.0963],
       grad_fn=<NllLossBackward0>) and loss is 6.720758438110352
losses are tensor([1.0831, 1.0763, 1.0342, 1.1134, 1.1159, 1.0458],
 

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relati

losses are tensor([1.1566, 1.1345, 1.1562, 1.0961, 1.1795, 1.2048],
       grad_fn=<NllLossBackward0>) and loss is 6.927670001983643
losses are tensor([1.1240, 1.0865, 1.1282, 1.0510, 1.1349, 1.1722],
       grad_fn=<NllLossBackward0>) and loss is 6.696740627288818
losses are tensor([1.0939, 1.0424, 1.1040, 1.0099, 1.0939, 1.1419],
       grad_fn=<NllLossBackward0>) and loss is 6.486083984375
losses are tensor([1.0676, 1.0018, 1.0828, 0.9728, 1.0562, 1.1149],
       grad_fn=<NllLossBackward0>) and loss is 6.296173572540283
losses are tensor([1.0427, 0.9653, 1.0626, 0.9400, 1.0216, 1.0891],
       grad_fn=<NllLossBackward0>) and loss is 6.121336460113525
losses are tensor([1.1515, 1.0647, 1.0750, 1.0913, 1.2529, 1.1209],
       grad_fn=<NllLossBackward0>) and loss is 6.756390571594238
losses are tensor([1.1054, 1.0234, 1.0363, 1.0406, 1.2179, 1.0784],
       grad_fn=<NllLossBackward0>) and loss is 6.502008438110352
losses are tensor([1.0620, 0.9838, 0.9988, 0.9926, 1.1843, 1.0380],
    

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [5]:
from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
from validation_datasets.ValidationDataset import ValidationDataset
import torch.utils.data as data

ds = ValidationDataset()

val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=2)
val_protomaml_loader = data.DataLoader(
    ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
)

# val_protomaml_loader = data.DataLoader(
#     None, batch_sampler=val_protomaml_sampler, num_workers=1
# )


In [6]:
for i in range(1):
    batch = next(iter(val_protomaml_loader))
#     print(batch)
    for episode_i in range(len(batch[0])):
        data, labels = batch[0][episode_i], batch[1][episode_i]
        supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2]
        print(supportLabels)

[1, 0, 1, 0]
[4, 4, 2, 2, 3, 3]
[5, 7, 6, 6, 5, 7]
[10, 8, 9, 9, 8, 10]
[12, 13, 11, 13, 11, 12]
[14, 15, 14, 15]
[17, 16, 17, 16]
[18, 18, 19, 19]
